# Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

default_path = '/content/drive/My Drive'

Mounted at /content/drive


# Change Directory

In [ ]:
%cd /content/drive/MyDrive/메타버스 아카데미/본과정/Python/data/dataset
%ls

/content/drive/MyDrive/메타버스 아카데미/본과정/Python/data/dataset
0/  0.csv  1/  1.csv  2/  2.csv  3/  3.csv  4/  4.csv  5/  5.csv


# Import

In [ ]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import pandas as pd
import numpy as np
import cv2

# Concat csv

In [ ]:
csv_paths = Path("./").glob("**/*.csv")
csv_list = [pd.read_csv(path) for path in csv_paths]

concated_df = pd.concat(csv_list)
concated_df.shape

(1247, 4)

In [ ]:
for item in concated_df:
    print(item)
    break

Unnamed: 0


# Split Data

In [ ]:
def data_split(dir_path:Path, df:pd.DataFrame):
    image_label_list = df["label"].unique()

    for label_name in image_label_list:
        image_path_list = df[df["label"]==label_name].loc[:63, ["label", "path"]]

        train_data, valid_data = train_test_split(image_path_list, test_size=0.2,shuffle=True, random_state=20230706)

        temp_dict = {
            'train': train_data,
            'valid': valid_data,
        }

        for task, data in temp_dict.items():
            copy_data(task, data)


def copy_data(task:str, data:pd.DataFrame):
    for idx, row in data.iterrows():
        label = row["label"]

        dataset_dir = Path("./").joinpath(f"dataset/{task}/{label}")
        dataset_dir.mkdir(parents=True, exist_ok=True)

        image_full_path = dataset_dir.joinpath(f"{label}_{idx}").with_suffix(".png")
        image = make_square_image(row["path"])

        if image is None:
            continue

        cv2.imwrite(str(image_full_path), image)


def make_square_image(image_path):
    origin_image = cv2.imread(image_path)
    origin_image = cv2.cvtColor(origin_image, cv2.COLOR_BGR2RGB)

    try:
        height, width, channels = origin_image.shape
    except:
        print(image_path, '#' * 10)
        return

    long_side = height if height > width else width

    squre_image = np.full((long_side, long_side, channels), origin_image.mean(), np.uint8)
    squre_image[int((long_side - height) / 2):int(long_side - (long_side - height) / 2),
                int((long_side - width) / 2):int(long_side - (long_side - width) / 2)] = origin_image

    return squre_image


In [ ]:
data_path = Path("./")
data_split(data_path, concated_df)